In [1]:
import numpy as np
from scipy import linalg as la


# Helper function for problems 1 and 2.
def index(A, tol=1e-5):
    """Compute the index of the matrix A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
        k (int): The index of A.
    """

    # test for non-singularity
    if not np.isclose(la.det(A), 0):
        return 0

    n = len(A)
    k = 1
    Ak = A.copy()
    while k <= n:
        r1 = np.linalg.matrix_rank(Ak)
        r2 = np.linalg.matrix_rank(np.dot(A,Ak))
        if r1 == r2:
            return k
        Ak = np.dot(A,Ak)
        k += 1

    return k


# Problem 1
def is_drazin(A, Ad, k):
    """Verify that a matrix Ad is the Drazin inverse of A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.
        Ad ((n,n) ndarray): A candidate for the Drazin inverse of A.
        k (int): The index of A.

    Returns:
        (bool) True of Ad is the Drazin inverse of A, False otherwise.
    """
    if np.allclose(A@Ad, Ad@A):         #check the first condition
        if np.allclose(np.linalg.matrix_power(A, k+1)@Ad, np.linalg.matrix_power(A, k)):      #check the second condition
            if np.allclose(Ad@A@Ad, Ad):             #check the third condition
                return True
    return False            #return false if any don't hold

def test1():
    A = np.array([[1,3,0,0], [0,1,3,0], [0,0,1,3], [0,0,0,0]])
    Ad = np.array([[1,-3,9,81], [0,1,-3,-18], [0,0,1,3],[0,0,0,0]])
    k = 1
    notA = np.array([[1,4,9,81], [0,1,-3,-18], [0,0,1,3],[0,0,0,0]])

    B = np.array([[1,1,3], [5,2,6], [-2,-1,-3]])
    Bd = np.zeros((3,3))
    k2 = 3
    notBd = np.identity(3)

    print(is_drazin(A, Ad, k))
    print(is_drazin(A, notA, k))
    print(is_drazin(B, Bd, 3))
    print(is_drazin(B, notBd, 3))
test1()

True
False
True
False


In [2]:
# Problem 2
def drazin_inverse(A, tol=1e-4):
    """Compute the Drazin inverse of A.

    Parameters:
        A ((n,n) ndarray): An nxn matrix.

    Returns:
       ((n,n) ndarray) The Drazin inverse of A.
    """
    n = A.shape[0]

    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol    #create our sorting functions

    T1, Q1, k1 = la.schur(A, sort=f)         #get Schur decomps
    T2, Q2, k2 = la.schur(A, sort=g)

    U = np.hstack((Q1[:,:k1], Q2[:,:n-k1]))         #create change of basis matrix

    Ui = la.inv(U)

    V = Ui@A@U              #find block diagonal matrix
    Z = np.zeros((n,n))

    if k1 != 0:
        Mi = la.inv(V[:k1, :k1])
        Z[:k1, :k1] = Mi

    return U@Z@Ui

def test2():
    A = np.array([[1,3,0,0], [0,1,3,0], [0,0,1,3], [0,0,0,0]])
    mydrazin = drazin_inverse(A)
    B = np.array([[1,1,3], [5,2,6], [-2,-1,-3]])
    myraisin = drazin_inverse(B)
    print(is_drazin(A, mydrazin, 1))
    print(is_drazin(B, myraisin, 3))
test2()

True
True


In [3]:
# Problem 3
def effective_resistance(A):
    """Compute the effective resistance for each node in a graph.

    Parameters:
        A ((n,n) ndarray): The adjacency matrix of an undirected graph.

    Returns:
        ((n,n) ndarray) The matrix where the ijth entry is the effective
        resistance from node i to node j.
    """
    n = A.shape[0]

    L = np.diag(np.sum(A, axis=0)) - A           #calculate the Laplacian

    R = np.zeros((n,n))

    iden = np.identity(n)

    for i in range(n):
        for j in range(n):
            if i!= j:
                L_ = np.copy(L)
                L_[j] = iden[j]             #replace the jth row of Laplacian with the jth row of the identity
                L_d = drazin_inverse(L_)        #find the Drazin inverse
                R[i,j] = L_d[i,i]               #build R

    return R

def test3():
    B = np.array([[0,1,0,0], [1,0,1,0], [0,1,0,1], [0,0,1,0]])
    print(effective_resistance(B))
test3()

[[0. 1. 2. 3.]
 [1. 0. 1. 2.]
 [2. 1. 0. 1.]
 [3. 2. 1. 0.]]


In [ ]:
# Problems 4 and 5
class LinkPredictor:
    """Predict links between nodes of a network."""

    def __init__(self, filename='social_network.csv'):
        """Create the effective resistance matrix by constructing
        an adjacency matrix.

        Parameters:
            filename (str): The name of a file containing graph data.
        """
        names = set()

        with open(filename, 'r') as myfile:
            myfile.readline()
            lines = [line for line in myfile]        #get a list of the lines

        for line in lines:
            fname, sname = line.strip().split(',')     #get the two names out of each line and add them to our set if they don't exist already
            names.add(fname)
            names.add(sname)

        names = list(names)
        n = len(names)

        adj_dict = dict(zip(names, np.arange(n)))     #create ordered dictionary of names to help build adjacency matrix

        Adj = np.zeros((n,n))

        for line in lines:
            fname, sname = line.strip().split(',')
            Adj[adj_dict[fname], adj_dict[sname]] = 1        #create edges going from first to second name

        Adj = Adj + Adj.T              #create edges going the other way

        self.names = names
        self.A = Adj                     #create and store each of the attributes
        self.R = effective_resistance(Adj)
        self.dict = adj_dict
        


    def predict_link(self, node=None):
        """Predict the next link, either for the whole graph or for a
        particular node.

        Parameters:
            node (str): The name of a node in the network.

        Returns:
            node1, node2 (str): The names of the next nodes to be linked.
                Returned if node is None.
            node1 (str): The name of the next node to be linked to 'node'.
                Returned if node is not None.

        Raises:
            ValueError: If node is not in the graph.
        """
        if node is None:
            not_con = self.R*(self.A == 0)          #zero out entries of resistance matrix that represent nodes already connected
            smallest = np.min(not_con[not_con > 0])      #find the smallest nonzero entry of the new resistance matrix
            loc = np.where(not_con==smallest)
            return self.names[loc[0][0]], self.names[loc[1][0]]       #find the names of the next two nodes to be linked

        elif node not in self.names:
            raise ValueError("This node is not in the network")        #raise a valuerror if the node isn't in the network

        else:
            node_row = self.dict[node]                           #only pay attention to the row corresponding to the given node
            not_con2 = self.R[node_row]*(self.A[node_row] == 0)     #zero entries where already connected
            minval2 = np.min(not_con2[not_con2>0])
            loc = np.where(not_con2==minval2)          #find the smallest nonzero entry of new matrix and get the corresponding node name to be connected

            return self.names[loc[0][0]]


    def add_link(self, node1, node2):
        """Add a link to the graph between node 1 and node 2 by updating the
        adjacency matrix and the effective resistance matrix.

        Parameters:
            node1 (str): The name of a node in the network.
            node2 (str): The name of a node in the network.

        Raises:
            ValueError: If either node1 or node2 is not in the graph.
        """
        if node1 not in self.names or node2 not in self.names:
            raise ValueError("At least one of these nodes is not in the network")           #raise a valuerror if either of the nodes is not in the network

        else:

            name1index = self.dict[node1]              #get the corresponding row/column for each node in the adjacency matrix
            name2index = self.dict[node2]

            self.A[name1index, name2index] += 1         #create an edge in the adjacency matrix both ways between the two nodes
            self.A[name2index, name1index] += 1
            self.R = effective_resistance(self.A)            #update the resistance matrix